In [55]:
import pandas as pd
fopen = open("20221001.as-rel2.txt", encoding="utf-8")

relationships = {}
degree_dict = {}
for line in fopen:
    if line[0] != "#":
        line_split = line.split("|")
        as1 = line_split[0].strip("\n")
        as2 = line_split[1].strip("\n")
        reltype = int(line_split[2].strip("\n"))
        if as1 not in relationships:
            # [customer array, provider array]
            relationships[as1] = [set(), set()]
            degree_dict[as1] = 0
        if as2 not in relationships:
            # [customer array, provider array]
            relationships[as2] = [set(), set()]
            degree_dict[as2] = 0
        # If p2c
        if reltype == -1:
            # as1 has customer
            relationships[as1][0].add(as2)
            # as2 has a provider
            relationships[as2][1].add(as1)
        degree_dict[as1] += 1
        degree_dict[as2] += 1

fopen.close()

# Find all AS's with no children
customers = {}
as_list = set()
for as_name in relationships:
    as_data = relationships[as_name]
    if len(as_data[0]) == 0:
        customers[as_name] = as_data[0]
        as_list.add(as_name)

for as_name in as_list:
    parents = relationships[as_name][1]
    for parent in parents:
        if parent in customers:
            customers[parent].add(as_name)
        else:
            customers[parent] = {as_name}
        as_list.add(as_name)
        for c in customers[as_name]:
            customers[parent].add(c)
            as_list.add(as_name)
sorted_customers = sorted(
    customers.items(), key=lambda key: len(key[1]), reverse=True)
total_as = 0
total_prefix = 0
total_ip = 0
as_stats = {}
for as_num in sorted_customers:
    as_stats[as_num[0]] = {}
    as_stats[as_num[0]]["as_number"] = len(as_num[1])
    total_as += len(as_num[1])

fopen = open("routeviews-rv2-20221001-1200.pfx2as", encoding="utf-8")

ipspace = {}

for line in fopen:
    if line[0] != "#":
        line_split = line.split()
        ip = line_split[0]
        prefix = int(line_split[1])
        as1 = line_split[2].strip("\n")
        if as1 not in ipspace:
            ipspace[as1] = [0, 0]
        ipspace[as1][0] += prefix
        ip_calc = 2 ** (32 - prefix)
        ipspace[as1][1] += ip_calc
        total_prefix += prefix
        total_ip += ip_calc
fopen.close()
for as_num in sorted_customers:
    if as_num[0] in ipspace:
        as_stats[as_num[0]]["prefix"] = ipspace[as_num[0]][0]
        as_stats[as_num[0]]["ip"] = ipspace[as_num[0]][1]
    else:
        as_stats[as_num[0]]["prefix"] = 0
        as_stats[as_num[0]]["ip"] = 0
org_to_name = {}
as_to_org = {}
fopen = open("20221001.as-org2info.txt", encoding="utf-8")
for line in fopen:
    if line[0] != "#":
        line_split = line.split("|")
        org_id = line_split[0]
        name = line_split[2]
        org_to_name[org_id] = name
    elif line.startswith("# format:aut"):
        for line in fopen:
            line_split = line.split("|")
            as_name = line_split[0]
            org_id = line_split[3]
            as_to_org[as_name] = org_id
fopen.close()
top_as = sorted_customers[:15]
top_dict = {}
for as_num in top_as:
    as_stats[as_num[0]]["name"] = org_to_name[as_to_org[as_num[0]]]
    as_stats[as_num[0]]["degree"] = degree_dict[as_num[0]]
    top_dict[as_num[0]] = as_stats[as_num[0]]
as_number = []
as_name = []
as_degree = []
ases = []
prefix = []
ips = []
perc_ases = []
perc_prefix = []
perc_ips = []
for as_num in top_dict:
    as_number.append(as_num)
    as_name.append(top_dict[as_num]["name"])
    as_degree.append(top_dict[as_num]["degree"])
    ases.append(top_dict[as_num]["as_number"])
    prefix.append(top_dict[as_num]["prefix"])
    ips.append(top_dict[as_num]["ip"])
    perc_ases.append(100 * top_dict[as_num]["as_number"] / total_as)
    perc_prefix.append(100 * top_dict[as_num]["prefix"] / total_prefix)
    perc_ips.append(100 * top_dict[as_num]["ip"] / total_ip)
table_dict = {"AS #": as_number, "AS name": as_name, "AS degree": as_degree, "ASes": ases,
              "IP Prefix": prefix, "IPs": ips, "ASes2": perc_ases, "IP Prefix2": perc_prefix, "IPs2": perc_ips}
table = pd.DataFrame(table_dict, index=list(range(1, 16)))
table.columns = [["AS #", "AS name", "AS degree", "customer cone", "customer cone", "customer cone", "customer cone", "customer cone", "customer"],
 ["", "", "", "number of", "number of", "number of", "percentage of", "percentage of", "percentage of"], 
 ["", "", "", "ASes", "IP Prefix", "IPs", "ASes", "IP Prefix", "IPs"]]
table


AS #                                 AS name AS degree customer cone  \
                                                                number of   
                                                                     ASes   
1    3356                     Level 3 Parent, LLC      6423          4564   
2     174                   Cogent Communications      6633          4470   
3    7018                     AT&T Services, Inc.      2458          2266   
4    6461                          Zayo Bandwidth      2322          1647   
5     701                        Verizon Business      1418          1241   
6    6939                  Hurricane Electric LLC      9766          1199   
7    3257                 GTT Communications Inc.      2014          1180   
8     209         CenturyLink Communications, LLC      1222          1111   
9    1299                        Telia Company AB      2335           967   
10   8220  COLT Technology Services Group Limited      2087           927   
11  46887         Lightower Fiber Networks I, LLC       948           890   
12   1239                                  Sprint      2143           874   
13  12389                         PJSC Rostelecom      1231           790   
14   3549                     Level 3 Parent, LLC      1085           788   
15   2914                       NTT America, Inc.      1590           765   

                                                     customer  
                        percentage of           percentage of  
   IP Prefix        IPs          ASes IP Prefix           IPs  
1      33463   37281120      3.952371  0.152388      0.883141  
2      75362   28753664      3.870968  0.343194      0.681137  
3      46636  115734273      1.962330  0.212377      2.741594  
4       4488    1564928      1.426283  0.020438      0.037071  
5      21931   44233984      1.074691  0.099872      1.047845  
6       3306     575232      1.038320  0.015055      0.013626  
7      24911    3676160      1.021866  0.113443      0.087083  
8      54630   28017920      0.962113  0.248781      0.663708  
9       2537     251136      0.837411  0.011553      0.005949  
10      3679    1270272      0.802771  0.016754      0.030091  
11     16203    1011968      0.770730  0.073787      0.023972  
12     32768   18498183      0.756874  0.149223      0.438198  
13     65200   16485376      0.684131  0.296917      0.390517  
14     26857    6020960      0.682399  0.122305      0.142629  
15      4838    6874368      0.662481  0.022032      0.162845